## 1. Подготовка к работе

In [1]:
from collections import defaultdict
import nltk
import pandas as pd
from torch.utils.data import Dataset
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import Trainer, TrainingArguments


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mdchumakov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##  2. Загрузка и предварительная обработка данных

### 2.1. Загрузка данных

In [3]:
df = pd.read_csv('data/work_data.csv')
df.head()

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3,Жилой комплекс,Московский квартал 2. Шумно : летом по ночам д...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. Дружелюбный персонал....
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   address  15000 non-null  object
 1   name_ru  15000 non-null  object
 2   rating   15000 non-null  int64 
 3   rubrics  15000 non-null  object
 4   text     15000 non-null  object
dtypes: int64(1), object(4)
memory usage: 586.1+ KB


### 2.2. Фильтрация и очистка данных

Возьмем данные, которые не содержат короткие отзывы (меньше 10 символов) и не содержат пропусков.

In [5]:
# Удаление отзывов с длиной текста менее 10 символов
df = df[df['text'].str.len() >= 10]
# Удаление пропущенных значений
df = df.dropna(subset=['text', 'name_ru', 'rating'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14998 entries, 0 to 14999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   address  14998 non-null  object
 1   name_ru  14998 non-null  object
 2   rating   14998 non-null  int64 
 3   rubrics  14998 non-null  object
 4   text     14998 non-null  object
dtypes: int64(1), object(4)
memory usage: 703.0+ KB


### 2.3. Преобразование категориальных данных

Для эффективного использования категорий и ключевых слов их необходимо включить в входной контекст модели.


In [6]:
# Кодирование рейтинга
df['rating'] = df['rating'].astype(int)

# Пример добавления ключевых слов (можно использовать TF-IDF или другие методы)
# Для простоты возьмем топ-N часто встречающихся слов в каждой рубрике
def get_top_n_keywords(df_: pd.DataFrame, n: int = 5) -> defaultdict:
    keywords = defaultdict(list)
    vectorizer = CountVectorizer(
        max_features=1000,
        stop_words=stopwords.words('russian'),
    )

    for rubric in df_['rubrics'].unique():
        texts = df_[df_['rubrics'] == rubric]['text']
        X = vectorizer.fit_transform(texts)
        counts = X.sum(axis=0).A1
        vocab = vectorizer.get_feature_names_out()
        top_n = [vocab[i] for i in counts.argsort()[-n:]]
        keywords[rubric] = top_n

    return keywords


keywords = get_top_n_keywords(df)
df['keywords'] = df['rubrics'].apply(lambda x: ', '.join(keywords.get(x, [])))
df.head()

,address,name_ru,rating,rubrics,text,keywords
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3,Жилой комплекс,Московский квартал 2. Шумно : летом по ночам д...,"дома, жк, рядом, это, очень"
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ...","вкусные, магазинов, магазин, блинчики, очень"
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я...","отличный, это, персонал, очень, зал"
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. Дружелюбный персонал....,"размер, редкий, сноуборда, условия, хорошие"
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...,"немного, жаль, щеточки, всё, топ"


### 2.4. Формирование входного текста для модели

Объединим все необходимые параметры в единый текстовый формат, который будет подаваться на вход модели.

In [7]:
def create_prompt(row):
    prompt = f"Категория: {row['rubrics']}\nРейтинг: {row['rating']}/5\nКлючевые слова: {row['keywords']}\nОтзыв:"
    return prompt

df['prompt'] = df.apply(create_prompt, axis=1)
df.head()

,address,name_ru,rating,rubrics,text,keywords,prompt
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3,Жилой комплекс,Московский квартал 2. Шумно : летом по ночам д...,"дома, жк, рядом, это, очень",Категория: Жилой комплекс\nРейтинг: 3/5\nКлюче...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ...","вкусные, магазинов, магазин, блинчики, очень",Категория: Магазин продуктов;Продукты глубокой...
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я...","отличный, это, персонал, очень, зал",Категория: Фитнес-клуб\nРейтинг: 1/5\nКлючевые...
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. Дружелюбный персонал....,"размер, редкий, сноуборда, условия, хорошие",Категория: Пункт проката;Прокат велосипедов;Са...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...,"немного, жаль, щеточки, всё, топ","Категория: Салон красоты;Визажисты, стилисты;С..."


### 2.5. Разделение данных на обучающую и тестовую выборки


In [8]:
train_df, eval_df = train_test_split(df, test_size=0.1, random_state=42)

## 3. Выбор и настройка модели

Используем предобученную модель GPT-2, адаптированную для русского языка. Для этого можно воспользоваться sberbank-ai/rugpt3small_based_on_gpt2 от SberBank.

In [9]:
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [10]:
special_tokens_dict = {'additional_special_tokens': ['Отзыв:']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(50258, 768)

## 4. Подготовка данных для обучения

Используем класс Dataset из PyTorch для создания подходящего формата данных.

In [11]:
class ReviewsDataset(Dataset):
    def __init__(self, prompts, texts, tokenizer, max_length=512):
        self.prompts = prompts
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.prompts)

    def __getitem__(self, idx):
        prompt = self.prompts[idx]
        text = self.texts[idx]
        encoding = self.tokenizer(prompt + ' ' + text, truncation=True, max_length=self.max_length, padding='max_length', return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        labels = input_ids.clone()
        # Мы хотим предсказывать только текст отзыва, поэтому маскируем промпт
        prompt_encoding = self.tokenizer(prompt, truncation=True, max_length=self.max_length, return_tensors='pt')
        labels[:prompt_encoding['input_ids'].size(1)] = -100  # Игнорируем потери для промпта

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }


In [12]:
train_dataset = ReviewsDataset(
    prompts=train_df['prompt'].tolist(),
    texts=train_df['text'].tolist(),
    tokenizer=tokenizer
)

eval_dataset = ReviewsDataset(
    prompts=eval_df['prompt'].tolist(),
    texts=eval_df['text'].tolist(),
    tokenizer=tokenizer
)


## 5. Обучение модели

Используем Trainer из Hugging Face для упрощения процесса обучения.

In [20]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=1000,
    eval_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=2,
    no_cuda=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()


/Users/mdchumakov/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1590: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,23.701200,22.121820
2,22.037700,22.694113
3,24.566100,24.259766


TrainOutput(global_step=10125, training_loss=24.0374500626929, metrics={'train_runtime': 36354.3918, 'train_samples_per_second': 1.114, 'train_steps_per_second': 0.279, 'total_flos': 1.0580759543808e+16, 'train_loss': 24.0374500626929, 'epoch': 3.0})

## 6. Генерация отзывов

После обучения модели можно использовать её для генерации новых отзывов на основе заданных параметров.

In [33]:
def generate_review(rubrics, rating, keywords, max_length=50):
    prompt = f"Категория: {rubrics}\nРейтинг: {rating}/5\nКлючевые слова: {keywords}\nОтзыв:"
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(
        input_ids,
        max_length=len(input_ids[0]) + max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        # early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    # Извлекаем только отзыв
    review = generated_text.split('Отзыв:')[-1].strip()
    return review

# Пример использования
new_review = generate_review(
    rubrics='Рестораны',
    rating=3,
    keywords="паста, атмосфера, обслуживание"
)
print(new_review)

Категория: Рестораны
Рейтинг: 3/5
Ключевые слова: паста, атмосфера, обслуживание
, , 
, Василий,стая, [, заклад, &,щая,[,ковая,&,ати, отли, сокру, Ваша, мягкая,://, вла, отмечает, Владислав, первая, всп, освящ, государственная, Вла


In [22]:
rubrics='Рестораны',
rating=4,
keywords='вкусно, обслуживание, уют'
prompt = f"Категория: {rubrics}\nРейтинг: {rating}/5\nКлючевые слова: {keywords}\nОтзыв:"
input_ids = tokenizer.encode(prompt, return_tensors='pt')
model.generate(input_ids)
# model

tensor([[ 6207,  4591,  1183,    30,   473,    11,  8706, 26322,   318, 22493,
            13,   203, 41342,  7851,    30,   473,    24,    16,    13,    19,
            25,   203,   684,   623,   390,  1936,  1694,    30, 24636,    16,
         16495,    16, 10360,   203, 50257,    16,   369,    16,   369,    16,
           369,    16,   369,    16,   369,    16,   369,    16,   369,    16,
           369,    16,   369,    16,   369]])

In [34]:
model.save_pretrained("models/model_v1")
tokenizer.save_pretrained("models/model_v1")

('models/model_v1/tokenizer_config.json',
 'models/model_v1/special_tokens_map.json',
 'models/model_v1/vocab.json',
 'models/model_v1/merges.txt',
 'models/model_v1/added_tokens.json')